# USAGE EXAMPLES FOR SIMPLESEDML

This notebook provides examples of using ``SimpleSBML``.

The initial emphasize is on time course simulations. A ``SingleModelTimeCourse`` object has one required parameter, the model. This can be an Antimony or SBML string or file or a URL. There are optional parameters related to the simulation and changing values of the paramters of the model. It has the following methods:
* ``getSEDML``
* ``getPhraSEDML``
* ``execute``: returns a ``DataFrame`` and a plot of the variables specified in the constructor.
* ``makeOMEXFile`` creates an omex file

A ``MultipleModelTimeCourseObject`` has the same arguments as for ``SingleModelTimeCourse`` except that the one required argument is a list of models. It has the same methods as well. The only difference is that there is one plot for each variable specified in the constructor with a curve for each model.

Some use cases are:
1. Iteratively develop SEDML by use execute, and then generate desired SEDML.
2. Generate PhraSEDML, edit it, and then generate SEDML from these edits.

In [1]:
# Source path (for github actions)
import sys
import os
project_path = os.path.dirname(os.path.dirname(os.path.abspath("__FILE__")))
src_path = os.path.join(project_path, "src")
sys.path.insert(0, src_path)
print(sys.path)

NameError: name 'os' is not defined

In [ ]:
# Handle execution locally and in Colab
try:
    import SimpleSEDML as ss
except:
    !pip install SimpleSEDML
    import SimpleSEDML as ss
import pandas as pd

In [ ]:
# Collect paths of files creates
MODEL_SOURCES = []

# Model

In [ ]:
MODEL_ANT = '''
model myModel
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    
    S1 = 10
    S2 = 0
    k1 = 1
    k2 = 1
end
'''

In [ ]:
MODEL2_ANT = '''
model myModel
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S1; k3*S3
    
    S1 = 10
    S2 = 0
    k1 = 1
    k2 = 1
    k3 = 0.1
end
'''

In [ ]:
IS_PLOT = True

In [ ]:
if IS_PLOT:
    dir(ss)

# Use Case 1: Single model with default arguments

In [ ]:
if IS_PLOT:
    ?ss.makeSingleModelTimeCourse

In [ ]:
smtc = ss.makeSingleModelTimeCourse(MODEL_ANT, title="My Plot", num_point=100, is_plot=IS_PLOT)

In [ ]:
if IS_PLOT:
    print(smtc.getSEDML())

We can also look at the PhraSED-ML that gets generated by changing the object that gets returned.

In [ ]:
if IS_PLOT:
    print(smtc.getPhraSEDML())

And we can execute the SEDML, getting both a plot and a DataFrame of time course values.

In [ ]:
df = smtc.execute()

In [ ]:
if IS_PLOT:
    df.head()

In [ ]:
MODEL_SOURCES.extend(smtc.model_sources)

# Use Case 2: Single model with non-default arguments

In [ ]:
WOLF_URL = "https://www.ebi.ac.uk/biomodels/services/download/get-files/MODEL3352181362/3/BIOMD0000000206_url.xml"

In [ ]:
smtc = ss.makeSingleModelTimeCourse(WOLF_URL, num_point=1000, is_plot=IS_PLOT)
MODEL_SOURCES.extend(smtc.model_sources)
_ = smtc.execute()

In [ ]:
if IS_PLOT:
    ss.getModelInformation(WOLF_URL)

In [ ]:
smtc = ss.makeSingleModelTimeCourse(WOLF_URL, k0=5, k1=55, title="Wolf with new parameter values",
        is_plot=IS_PLOT)
df = smtc.execute()

In [ ]:
if IS_PLOT:
    df.head()

In [ ]:
MODEL_SOURCES.extend(smtc.model_sources)

# Use Case 3: Comparing the time course simulation of multiple models

When comparing the time course simulation of multiple models, we want to compare selected **display variables** from the models, either in plots or in a DataFrame. This is done with the  ``MultipleModelTimeCourse`` object.

In [ ]:
mmtc = ss.makeMultipleModelTimeCourse([MODEL_ANT, MODEL2_ANT], k1=10, num_point=100,
                                     is_plot=IS_PLOT)

In [ ]:
MODEL_SOURCES.extend(mmtc.model_sources)
if IS_PLOT:
    mmtc.execute()

In [ ]:
if IS_PLOT:
    print(mmtc.getPhraSEDML())

In [ ]:
if IS_PLOT:
    print(mmtc.getSEDML())

## Creating an OMEX file

We can create an OMEX file for this experiment.

In [ ]:
omex_path, _ = mmtc.makeOMEXFile()
omex_path

In [ ]:
if IS_PLOT:
    !unzip -l project.omex

In [ ]:
MODEL_SOURCES.extend(mmtc.model_sources)

# Under construction: Compare several parameters at steady state for a single model

# Under construction: Compare stochastic simulations

# Under construction: Get data for several different simulations

In [ ]:
# Compose an experiment
ssed = ss.SimpleSEDML()
ssed.addModel("model1", MODEL_ANT, parameter_dct=dict(k1=3), ref_type="ant_str", is_overwrite=True)
ssed.addSimulation("sim1", "uniform", start=0, end=1, num_step=50)
ssed.addTask("task1", "model1", "sim1")
ssed.addReport("time", "S1", "S2", "S3")
ssed.addPlot("time", "S2", title="my plot", is_plot=IS_PLOT)

In [ ]:
# Run the descript
df = ssed.execute()

In [ ]:
if IS_PLOT:
    df.head(10)

In [ ]:
ssed.getAllModelInformation()

In [ ]:
MODEL_SOURCES.extend(smtc.model_sources)

# Under construction: Repeated Tasks

At present, the Tellurium execution of SED-ML only provides access to the last report for repeated tasks.

In [ ]:
# Make DataFrame of changes to model parameters
parameter_df = pd.DataFrame(dict(k1=[1, 3], k2=[1, 3]))

In [ ]:
ssed = ss.SimpleSEDML()
ssed.addModel("model1", MODEL_ANT, parameter_dct=dict(k1=3), ref_type="ant_str", is_overwrite=True)
ssed.addSimulation("sim1", "uniform", start=0, end=1, num_step=50)
ssed.addTask("task1", "model1", "sim1")
ssed.addRepeatedTask("repeated_task1", "task1", parameter_df=parameter_df)
ssed.addReport("task1.time", "task1.S1", "task1.S2", "task1.S3")
ssed.addPlot("repeated_task1.time", "repeated_task1.S2", title="my plot", is_plot=IS_PLOT)
result_df = ssed.execute()

In [ ]:
if IS_PLOT:
    print(ssed.getSEDML())

In [ ]:
MODEL_SOURCES.extend(ssed.model_sources)

# Cleanup

In [ ]:
import os
EXAMPLE_REMOVE_FILES = ["simple_sedml_plot.png", "project.omex", "project.omex"]
example_dir = os.getcwd()
remove_files = [os.path.join(example_dir, f) for f in EXAMPLE_REMOVE_FILES]
remove_files.extend(MODEL_SOURCES)
for ffile in remove_files:
    if os.path.isfile(ffile):
        os.remove(ffile)